In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# # 최신버전 1.x 대응 임포트 TODO [20260114@terry007x] langchain 1.2.3 에서 module import 안 됨... AgentType을 String 값으로 지정 agent_type="zero-shot-react-description",
# try:
#     from langchain.agents.agent_types import AgentType
# except ImportError:
#     # 혹시 모를 하위 호환성 위한 예외 처리... 미치겠다...
#     from langchain.agents import AgentType

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/중도탈락 학생 현황 (대학)_2026-01-14175237927.csv" #
df = pd.read_csv(file_path, encoding='utf-8')  # 'cp949')

print("데이터 로드 완료")
display(df.head()) 
print(len(df))

# 3. 모델 및 에이전트 설정
model_name = "gemma3:27b"  # "gemma3:27b"
"""
mixtral: \ 문제
"""
llm = OllamaLLM(model=model_name, 
                temperature=0,
                # 모델이 한 번의 응답에서 생성할 최대 토큰 수 (설명 길이를 강제 제한) 길게 설명하는 것을 방지
                num_predict=2048,  # # 512,  # 이상한 소리하면 줄여보기
               )
llm_with_stop = llm.bind(stop=["Observation:", "\nObservation", "Thought:", "\n\tObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = """
You are a specialized Python data analysis expert.
Your ONLY goal is to solve the user's request using the dataframe `df`.
Before running any analysis, ensure you use 'import pandas as pd' and 'import numpy as np' if needed.
You must use the following format:

[CRITICAL RULE]
1. A pandas dataframe named `df` is ALREADY LOADED in your environment.
2. DO NOT create a new dataframe. DO NOT use `pd.DataFrame()`.
3. ONLY use the existing `df` variable.
4. If you need to know the structure, use `df.head()` or `df.info()`.
5. Your output must strictly follow the Thought/Action/Action Input format. 
6. Do not give advice or explanations outside the format.
7. NEVER explain how functions (like idxmax, groupby) work. 
8. Your output must ALWAYS start with 'Thought:'.
9. After 'Action Input:', you must stop and wait for 'Observation:'.
10. Only provide the final report in the 'Final Answer' section, not during the process.
11. Once you have all the information, you MUST provide the final report.
12. The report MUST start with the EXACT string 'Final Answer: ' followed by the delimiter.

[STRICT FORMATTING RULES]
1. You MUST follow the ReAct format: Thought, Action, Action Input, Observation.
2. The ONLY valid Action is: python_repl_ast
3. Action Input MUST be RAW Python code. 
   - DO NOT surround code with backticks (```).
   - DO NOT use any markdown formatting inside Action Input.
4. Stop speaking immediately after providing the Action Input.

[EXAMPLE]
Thought: I need to check the average temperature.
Action: python_repl_ast
Action Input: df['TEMP'].mean()
Observation: (wait for this)

[STRICT REPETITION PREVENTION RULES]
1. Before taking an Action, review your previous Thoughts and Observations.
2. If you have already obtained the result (Observation), DO NOT run the same code again.
3. Once you get the statistical data, move on to the next step: Interpretation or Visualization.
4. If you have enough information to answer the user's request, provide the 'Final Answer' immediately.

[CRITICAL RULE FOR PANDAS]
1. NEVER use `numeric_only=True` inside the `df.describe()` function. It will cause a TypeError.
2. To describe only numeric columns, use `df.describe(include=[np.number])` or `df.select_dtypes(include=[np.number]).describe()`.
3. When using aggregation functions like .mean(), .sum(), or .std(), ALWAYS use the argument `numeric_only=True`.
   Example: df.mean(numeric_only=True)
4. If a TypeError occurs during calculation, check the data types using df.dtypes and convert them to numeric using pd.to_numeric(errors='coerce') if necessary.

[Current Dataframe Info]
- Variable Name: df
- Columns: {col_list}

[VISUALIZATION RULE]
- If you create a graph, you MUST execute:
  plt.savefig('chart.png')
  plt.show()
- This ensures the file is available for the report and visible to the user.

[STRICT RULES FOR FINAL ANSWER]
1. NEVER use the LaTeX format 'The final answer is $\\boxed$'.
2. NEVER leave the Final Answer empty.
3. NEVER include Python code in the 'Final Answer'.
4. The 'Final Answer' must be a polished report written in KOREAN.
5. You must use the values obtained from the 'Observation' steps.
6. If you created a chart, mention the filename (e.g., 'analysis_plot.png').
7. Use the following format for your 'Final Answer:'.

[REPORT FORMAT 예시]
Final Answer:
==================================================
            분석 결과 보고서
==================================================
1. 분석 요약: (전체적인 분석 결과를 한 줄 요약)

2. 주요 통계 수치:
   (평균값, 분산, 상관분석, 분산분석, 회귀분석, 요인분석, 주성분분석)

3. 도표 (Visual Analysis):
   - 분석 결과 그래프를 'chart.png'로 저장하고 화면에 출력하였습니다.

4. 데이터 상세 분석:
   - (데이터의 흐름과 특징을 구체적인 숫자를 인용하여 설명)

5. 비즈니스 제언:
   - (분석 결과를 바탕으로 서비스 운영에 도움이 될 만한 아이디어)
==================================================
""".format(col_list=list(df.columns))

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력
    allow_dangerous_code=True,  # 코드 실행 허용
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리 (에러 발생 시 모델에게 던져줄 가이드 문구)
    agent_executor_kwargs={
        "handle_parsing_errors": (
            "Your Final Answer was invalid. "
            "DO NOT use '$\\boxed{}$'. "
            "NEVER use the LaTeX format 'The final answer is $\\boxed{}$'."
            "You MUST start your Final Answer with the report delimiter '==================================================' "
            "and follow the [REPORT FORMAT] exactly in KOREAN."
            "Check your format. Ensure 'Action: python_repl_ast' is present "
            "and 'Action Input' contains ONLY raw Python code without any backticks or markdown tags."
            "ERROR: You provided a conversational response instead of a structured Action. "
            "You MUST use the format: 'Thought: <reasoning>\nAction: python_repl_ast\nAction Input: <code>'. "
            "Do not explain functions. Just execute the code for the given 'df'."
            "If you get a NameError like 'np is not defined', "
            "always start your code with 'import numpy as np' in the Action Input."
            "분석이 완료되었다면 반드시 'Final Answer: '라는 단어로 답변을 시작하십시오. "
            "예시: Final Answer: =========================..."
        ),
    },
    agent_type="zero-shot-react-description",
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 명시적 타입 지정
    max_iterations=100, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 통계값 요약해줘"
hypo_req = "학업중단 사유별 학업중단율을 비교하고 가설을 설정하여라"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.

주의사항:
- 첫 번째 줄은 반드시 'Thought:'로 시작하세요.
- 설명을 하거나 인사말을 하지 마세요.
- 불필요한 서술이나 코드에 대한 설명(예: '이 함수는 ~입니다', '수정한 후 결과는 ~')을 절대 하지 마세요.
- 파이썬 코드를 실행해야 할 때는 반드시 'Action: python_repl_ast' 형식을 사용하고, 코드 블록(```python)을 직접 답변에 포함하지 마세요.
- 당신의 답변은 반드시 'Thought:', 'Action:', 'Action Input:', 'Observation:'의 형식을 엄격히 따라야 합니다.
- 평균(mean)이나 합계(sum)를 구할 때는 반드시 숫자형 컬럼만 선택하거나 `numeric_only=True` 옵션을 사용하세요.
  예: df.groupby('col').mean(numeric_only=True)
- 데이터에 문자열이나 날짜가 섞여 있을 수 있음을 항상 유의하세요.
- 시각화는 matplotlib이나 seaborn을 사용하세요.
- 한글 폰트(NanumGothic)가 이미 설정되어 있으니 그대로 사용하세요.
- 주피터 노트북 환경이므로 별도의 파일 저장 없이 plt.show()를 호출하여 그래프를 출력하세요.
- 코드 수정 제안이나 인사말 등 불필요한 텍스트를 출력하지 마세요. 바로 분석에 필요한 코드를 실행하세요.
- 분석 결과(Observation)를 얻기 전까지는 'Final Answer:'를 내지 마세요.
- 최종 답변을 낼 때만 'Final Answer:'를 사용하세요.
- 모든 설명은 한국어로 작성하세요.

[실행 규칙]
- Action: 반드시 'python_repl_ast'라고 적으세요.
- Action Input: markdown backticks(```)을 절대 사용하지 마세요. 
- 예시:
Action: python_repl_ast
Action Input: print(df.head())

[REPORTING INSTRUCTIONS]
1. 분석을 마친 후, 반드시 위에서 정의한 [REPORT FORMAT]에 맞춰 한국어로 보고서를 작성하세요.
2. 영어 답변은 절대로 허용하지 않습니다.
3. 통계 수치 뒤에는 반드시 단위를 붙이세요. (분, km 등)
"""

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.run(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

데이터 로드 완료


,기준연도,학교종류,설립구분,지역,상태,학교,재적학생\n(A),계(B),미등록,미복학,...,계(B`),미등록.1,미복학.1,자퇴.1,학사경고.1,학생활동.1,유급제적.1,재학연한초과.1,기타.1,중도탈락학생(신입생)비율(%)\n(B`/A`) × 100
0,2024,대학교,사립,경남,기존,가야대학교(김해),"2,012",179,14,30,...,42,2,0,40,0,0,0,0,0,11.4
1,2024,대학교,사립,경기,기존,가천대학교,"28,264","1,090",133,117,...,451,14,0,437,0,0,0,0,0,8.6
2,2024,대학교,사립,강원,기존,가톨릭관동대학교,"8,410",627,4,183,...,127,1,0,126,0,0,0,0,0,10.8
3,2024,대학교,사립,충북,기존,가톨릭꽃동네대학교,557,26,0,0,...,18,0,0,14,0,0,0,0,4,15.9
4,2024,대학교,사립,경기,기존,가톨릭대학교,"9,794",384,15,61,...,164,3,0,161,0,0,0,0,0,8.8


246
gemma3:27b AGENT GO!GO!GO!...



> Entering new AgentExecutor chain...


/tmp/ipykernel_240911/3162879297.py:209: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  response = agent.run(full_query)


Thought: I need to get the descriptive statistics of the dataframe.
Action: python_repl_ast
Action Input: df.describe(include=[np.number])NameError: name 'np' is not definedThought: I need to import numpy to use it in the describe function.
Action: python_repl_ast
Action Input: import numpy as np; df.describe(include=[np.number])         기준연도         미복학        학사경고        학생활동        유급제적      재학연한초과  \
count   246.0  246.000000  246.000000  246.000000  246.000000  246.000000   
mean   2024.0  105.658537    7.504065    0.040650    0.166667    1.199187   
std       0.0  119.074069   12.082623    0.197882    0.917331    6.429282   
min    2024.0    0.000000    0.000000    0.000000    0.000000    0.000000   
25%    2024.0   13.250000    0.000000    0.000000    0.000000    0.000000   
50%    2024.0   74.000000    1.000000    0.000000    0.000000    0.000000   
75%    2024.0  154.500000   10.000000    0.000000    0.000000    0.000000   
max    2024.0  889.000000   65.000000    1.000000   1

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama   # # from langchain_ollama import OllamaLLM  # # TODO 차후 최신 OllamaLLM 로 변경 필요
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# from langchain.agents import AgentType  # # from langchain.agents.agent_types import AgentType

# 1. 로컬 LLM 설정
# [중요] Agent가 정확한 코드 생성위해 temperature=0 설정
llm = Ollama(model="mistral-small", temperature=0)
# llm = OllamaLLM(model="mistral-small", temperature=0)

# 2. 분석위한 dummy data TODO csv나 DB에서 실데이터 불러오는 것으로 수정
data = {
    'date': pd.date_range(start='2025-10-01', periods=14),
    'visitor_count': [1200, 1300, 1100, 900, 850, 1500, 1600, 1150, 1050, 950, 800, 1450, 1550, 1100],
    'order_count': [120, 135, 115, 80, 75, 180, 195, 110, 100, 85, 70, 175, 190, 105],
    'ad_spend': [500, 500, 500, 300, 300, 700, 700, 500, 500, 500, 300, 700, 700, 500],
    'platform': ['PC', 'Mobile', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC']
}
df = pd.DataFrame(data)

# 3. Pandas DataFrame Agent 생성
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True, # Agent가 생각 과정(Thought/Action) 화면 출력
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # # 로컬 환경에서 LLM이 python 실행 허용 설정
)

# 4. 분석 요청 프롬프트 (Few-shot 구조 결합)
query = """
제공된 데이터프레임을 기반으로 다음 구조에 맞춰 한국어 보고서를 작성하세요:

1. 데이터 요약: 전체적인 수치(방문자, 주문수 등) 요약
2. 가설 설정: 플랫폼(PC/Mobile)별 효율 차이나 요일별 특징에 대한 가설
3. 분석 결과: 가설을 검증하기 위해 실제 코드를 실행한 통계적 근거 (예: 플랫폼별 전환율 비교)
4. 비즈니스 제언: 분석 결과를 바탕으로 한 마케팅 전략

반드시 코드를 실행하여 정확한 평균값과 상관관계를 계산한 뒤 답변하세요.
"""

print("Agent GO!GO!GO!...")
print("-" * 50)

try:
    response = agent.run(query)
    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")